In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data)
market_data_scaled[:5]

In [ ]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original DataFrame
# Set the coin_id column as index
scaled_df = pd.DataFrame(market_data_scaled, columns=df_market_data.columns, index=df_market_data.index)

# Display the scaled DataFrame
scaled_df.head()

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))
k_values

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(market_data_scaled)
    inertia.append(model.inertia_)

inertia

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
curve_dict = {"k": k_values, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
plot_df = pd.DataFrame(curve_dict)

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
plot_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_values,
    xlabel="Number of clusters",
    ylabel="Inertia",
    width=800,
    height=400
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4 is the best value for K as it is where we see the last big change in slope.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [ ]:
# Fit the K-Means model using the scaled DataFrame
model.fit(market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predictions = model.fit_predict(scaled_df)

# Print the resulting array of cluster values.
print(predictions)

In [ ]:
# Create a copy of the scaled DataFrame
scaled_df_copy = scaled_df.copy()


In [ ]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
scaled_df_copy["predicted clusters"] = predictions.tolist()

# Display the copy of the scaled DataFrame
scaled_df_copy.head()

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scatter_plot = scaled_df_copy.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='predicted clusters',  # Colors points by KMeans cluster labels
    hover_cols=['coin_id'],  # Shows crypto name on hover
    height=400,
    width=600,
    title='Cryptocurrency Price Changes: 24h vs 7d',
    xlabel='24h Price Change (%)',
    ylabel='7d Price Change (%)'
)

# Display the plot
scatter_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca.fit_transform(scaled_df)

# Create DataFrame with PCA results
pca_df = pd.DataFrame(
    data=pca_data,
    columns=['PC 1', 'PC 2', 'PC 3'],
    index=scaled_df.index
)

# View the scaled PCA data
pca_df.head()


In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
# Calculate explained variance ratio and convert to percentages
explained_variance = pca.explained_variance_ratio_
explained_variance_pct = [f"{var:.2%}" for var in explained_variance]

# Display the explained variance for each principal component
print("Explained Variance Ratio:")
for i, variance in enumerate(explained_variance_pct, 1):
    print(f"Principal Component {i}: {variance}")

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5% of all variance is explained by the 3 principal componets

In [ ]:
# Create a new DataFrame with the PCA data.
pca_df_new = pd.DataFrame(
    pca_data, # Defined in box [62]
    columns=['PC 1', 'PC 2', 'PC 3'],
    index=scaled_df.index
)

# Copy the crypto names from the original scaled DataFrame
# Set the coin_id column as index
pca_df_new['coin_id'] = scaled_df.index
pca_df_new.reset_index(drop=True, inplace=True)
pca_df_new.set_index('coin_id', inplace=True)

# Display the scaled PCA DataFrame
pca_df_new.head()

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [ ]:
# Create a list with the number of k-values from 1 to 11
print(k_values)

In [ ]:
# Create an empty list to store the inertia values
intertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(pca_df_new)
    intertia.append(model.inertia_)

intertia


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
crpyto_curve_dict = {"k": k_values, "inertia": intertia}
# Create a DataFrame with the data to plot the Elbow curve
crypto_df = pd.DataFrame(crpyto_curve_dict)
crypto_df.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
crypto_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_values,
    xlabel="Number of clusters",
    ylabel="Inertia",
    width=800,
    height=400
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k is 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the best value for k did not change from the original data

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
# Initialize K-Means model with optimal k=4
model = KMeans(n_clusters=4, random_state=1)


In [ ]:
# Fit the K-Means model using the PCA data


In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame


# Print the resulting array of cluster values.


In [ ]:
# Create a copy of the scaled PCA DataFrame


# Add a new column to the copy of the PCA DataFrame with the predicted clusters


# Display the copy of the scaled PCA DataFrame


In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.


### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
# test

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 